In [1]:
# import os
# from neo4j import GraphDatabase as graphdb
# from trengx import graph

import os
from dotenv import load_dotenv
from neo4j import GraphDatabase as graphdb
import sys
sys.path.append('../trengx')
import graph

In [2]:
# .env secrets
uri = os.getenv('NEO4J_URI') # Get the value of the uri variable
user = os.getenv('NEO4J_USER') # Get the value of the user variable
password= os.getenv('NEO4J_PASSWORD')  # Get the value of the pw variable

# Create a Graph class instance
g = graph.Graph(graphdb, uri, user, password)


In [46]:
g.delete_all()

# y_est[i] = w_est[i]*y[i-1]
w_est_i = g.add_node ('num', 'w_est_i', {'value': 0.1})
w_est_i_id = w_est_i['node_id']
y_im1 = g.add_node ('num', 'y_im1 ', {'value': 10})
y_im1_id = y_im1['node_id']
mul = g.add_node ('op', '*', {'reverse':False})
mul_id = mul['node_id']
g.add_edge ('num2op', w_est_i_id, mul_id, {'trigger':False})
g.add_edge ('num2op', y_im1_id, mul_id, {'trigger':True})
y_est_i = g.add_node ('num', 'y_est_i', {'value': 0})
y_est_i_id = y_est_i['node_id']
g.add_edge ('op2num', mul_id, y_est_i_id, {'trigger':True})

# e[i] = y[i] - y_est[i]
y_i = g.add_node ('num', 'y_i', {'value': 10})
y_i_id = y_i['node_id']
sub = g.add_node ('op', '-', {'reverse':True})
sub_id = sub['node_id']
g.add_edge ('num2op', y_i_id, sub_id, {'trigger':False})
g.add_edge ('num2op', y_est_i_id, sub_id, {'trigger':True})
e_i = g.add_node ('num', 'e_i', {'value': 0.2})
e_i_id = e_i['node_id']
g.add_edge ('op2num', sub_id, e_i_id, {'trigger':True})

# w_est[i+1] = w_est[i] + lr*y[i-1]*e[i]
mul = g.add_node ('op', '*', {'reverse':False})
mul_id = mul['node_id']
g.add_edge ('num2op', y_im1_id, mul_id, {'trigger':False})
g.add_edge ('num2op', e_i_id, mul_id, {'trigger':True})
temp1 = g.add_node ('num', 'temp1', {'value': 1.0})
temp_id_1 = temp1['node_id']
g.add_edge ('op2num', mul_id, temp_id_1, {'trigger':True})
lr = g.add_node ('num', 'lr', {'value': 0.001})
lr_id = lr['node_id']
mul = g.add_node ('op', '*', {'reverse':False})
mul_id = mul['node_id']
g.add_edge ('num2op', temp_id_1, mul_id,{'trigger':True})
g.add_edge ('num2op', lr_id, mul_id, {'trigger':False})
temp2 = g.add_node ('num', 'temp2', {'value': 1.0})
temp_id_2 = temp2['node_id']
g.add_edge ('op2num', mul_id, temp_id_2, {'trigger':True})
add = g.add_node ('op', '+', {'reverse':False})
add_id = add['node_id']
g.add_edge ('num2op', w_est_i_id, add_id, {'trigger':False})
g.add_edge ('num2op', temp_id_2, add_id, {'trigger':True})
w_est_ip1 = g.add_node ('num', 'w_est_ip1', {'value': 0.2})
w_est_ip1_id = w_est_ip1['node_id']
g.add_edge ('op2num', add_id, w_est_ip1_id, {'trigger':True})


[{'edge_id': 13,
  'edge_label': 'op2num',
  'out_node_id': 122,
  'in_node_id': 123,
  'properties': {'trigger': True}}]

In [48]:
query = """
MATCH path = (in1)-[r:num2op|op2num*0..]->()
WHERE id(in1) = $node_id AND all(rel IN relationships(path) WHERE rel.trigger = true)
WITH path
ORDER BY length(path) DESC
LIMIT 1
WITH nodes(path) AS nodes
UNWIND range(0, size(nodes)-2, 2) AS i
WITH nodes[i] AS in1, nodes[i+1] AS op, nodes[i+2] AS out
MATCH (in1)-[:num2op]->(op)-[:op2num]->(out)
OPTIONAL MATCH (in2)-[:num2op]->(op)
WHERE id(in2) <> id(in1)  
WITH in1, op, out, in2
SET out.value = 
  CASE 
    WHEN op.name = '+' THEN in1.value + in2.value
    WHEN op.name = '-' AND op.reverse = false THEN in1.value - in2.value
    WHEN op.name = '-' AND op.reverse = true THEN in2.value - in1.value
    WHEN op.name = '*' THEN in1.value * in2.value
    WHEN op.name = '/' AND op.reverse = false THEN in1.value / in2.value
    WHEN op.name = '/' AND op.reverse = true THEN in2.value / in1.value
    WHEN op.name = 'round' THEN round(in1.value)
    ELSE out.value
  END
RETURN out
"""
params = {"node_id": y_im1_id}  
g.run_query(query, params)
print (g.get_node_prop(y_est_i_id, 'value'))
print (g.get_node_prop(e_i_id, 'value'))
print (g.get_node_prop(temp_id_1, 'value'))
print (g.get_node_prop(temp_id_2, 'value'))
print (g.get_node_prop(w_est_ip1_id, 'value'))


1.0
9.0
90.0
0.09
0.19


In [54]:
g.delete_all()

# y_est[i] = w_est[i]*y[i-1]
a = g.add_node ('num', 'a', {'value': 9})
a_id = a['node_id']
b = g.add_node ('num', 'b', {'value': 10})
b_id = b['node_id']
round = g.add_node ('op', 'sqrt', {'reverse':False})
round_id = round['node_id']
g.add_edge ('num2op', a_id, round_id, {'trigger':True})
g.add_edge ('op2num', round_id, b_id, {'trigger':True})


[{'edge_id': 2,
  'edge_label': 'op2num',
  'out_node_id': 132,
  'in_node_id': 131,
  'properties': {'trigger': True}}]

In [53]:
query = """
MATCH path = (in1)-[r:num2op|op2num*0..]->()
WHERE id(in1) = $node_id AND all(rel IN relationships(path) WHERE rel.trigger = true)
WITH path
ORDER BY length(path) DESC
LIMIT 1
WITH nodes(path) AS nodes
UNWIND range(0, size(nodes)-2, 2) AS i
WITH nodes[i] AS in1, nodes[i+1] AS op, nodes[i+2] AS out
MATCH (in1)-[:num2op]->(op)-[:op2num]->(out)
OPTIONAL MATCH (in2)-[:num2op]->(op)
WHERE id(in2) <> id(in1)  
WITH in1, op, out, in2
SET out.value = 
  CASE 
    WHEN op.name = '+' THEN in1.value + in2.value
    WHEN op.name = '-' AND op.reverse = false THEN in1.value - in2.value
    WHEN op.name = '-' AND op.reverse = true THEN in2.value - in1.value
    WHEN op.name = '*' THEN in1.value * in2.value
    WHEN op.name = '/' AND op.reverse = false THEN in1.value / in2.value
    WHEN op.name = '/' AND op.reverse = true THEN in2.value / in1.value
    WHEN op.name = 'round' THEN round(in1.value)
    WHEN op.name = 'abs' THEN abs(in1.value)
    WHEN op.name = 'exp' THEN exp(in1.value)
    WHEN op.name = 'log' THEN log(in1.value)
    WHEN op.name = 'log10' THEN log10(in1.value)
    WHEN op.name = 'sqrt' THEN sqrt(in1.value)
    WHEN op.name = 'sin' THEN sin(in1.value)
    WHEN op.name = 'cos' THEN cos(in1.value)
    WHEN op.name = 'tan' THEN tan(in1.value)
    ELSE out.value
  END
RETURN out
"""

params = {"node_id": a_id}  
g.run_query(query, params)
print (g.get_node_prop(b_id, 'value'))

8.0
